<a href="https://colab.research.google.com/github/MathMachado/Python_RFB/blob/master/Projetos/Titanic/3DP_Feature%20Engineering_Fase1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PROJETO TITANIC
## 3DP - DATA PREPARATION
## 3DP - Feature Engineering - Fase 1

* Objetivo
    * Aplicar Machine Learning para problemas de classificação

# Machine Learning com Python (Scikit-Learn)

![Scikit-Learn](https://github.com/MathMachado/Python_RFB/blob/master/Material/scikit-learn-1.png?raw=true)

# Dados

* Dicionário de dados do dataframe Titanic:
    * **PassengerID**: ID do passageiro;
    * **Survived**: Indicador, sendo 1= Passageiro sobreviveu e 0= Passageiro morreu;
    * **Pclass**: Classe em que o passageiro viaja (1 classe, 2 classe, 3 classe, etc);
    * **Age**: Idade do Passageiro;
    * **SibSp**: Número de parentes a bordo (esposa, irmãos, pais e etc);
    * **Parch**: Número de pais/crianças a bordo;
    * **Fare**: Valor pago pela viagem;
    * **Cabin**: Cabine do Passageiro;
    * **Embarked**: A porta pelo qual o Passageiro embarcou.
    * **Name**: Nome do Passageiro;
    * **Sex**: Sexo do Passageiro.

# Carregar as bibliotecas (genéricas) Python

In [0]:
# Pandas
import pandas as pd
import numpy as np
from pandas import Series, DataFrame

# matplotlib, seaborn
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

# Carregar Dados

In [0]:
url_train= 'https://raw.githubusercontent.com/MathMachado/Python_RFB/master/Dataframes/Titanic_With_MV.csv?token=AGDJQ6YNSL7NCYOKQT3FTGK5NV3WQ'
url_test= 'https://raw.githubusercontent.com/MathMachado/Python_RFB/master/Dataframes/Titanic_test.csv?token=AGDJQ622BQZRXWZII2SCCWK5NV3X4'

# Carrega os dataframes de treinamento e teste e define 'PassengerId' como chave
df_train= pd.read_csv(url_train, index_col='PassengerId')
df_test= pd.read_csv(url_test, index_col='PassengerId')

# Faz uma cópia dos dados originais da variável resposta 'Survived'
df_train_Survived = df_train["Survived"].copy()

# merge train and test
df = df_train.append(df_test, sort= False)

# Entendendo o dataframe

Inicialmente, vamos avaliar os dataframes df_train e df_test:

In [0]:
df_train.shape

In [0]:
df_test.shape

* Quantas linhas e colunas possui o dataframe de treinamento e teste?
* Porque eu juntei os dois dataframes (df_train e df_test) num único dataframe?

Abaixo, listamos a lista das variáveis do dataframe:

In [0]:
df.columns

Quantas linhas e colunas tem noso dataframe?

In [0]:
df.shape

* Quantas linhas e colunas possuem nosso dataframe?

Visualizar (parte) do dataframe:

In [0]:
df.head(50)

O comando a seguir apresenta as principais informações acerca do dataframe:

In [0]:
df.info() 

# Deletar variáveis/atributos que não serão usados na modelagem
* Observe que a variável 'Ticket' não tem valor algum para nosso propósito. Por conta disso, vamos deletá-lo.

Inicialmente, vamos deletar as variáveis que não são de interesse para a análise. Isso nos ajuda a focar no que de fato é importante.

In [0]:
df = df.drop(['Ticket'], axis=1) # axis= 1 indica que se trata de uma operação na coluna do dataframe. Lembre-se: axis= 0 indica operação nas linhas do dataframe.
df.head(3)

Observe que a coluna 'Ticket' foi de fato deletada do dataframe.

A seguir, crio a variável 'Survived2' para ajudar no entendimento dos dados:

In [0]:
df['Survived2'] = df['Survived']
df['Survived2'] = df['Survived2'].map({0:'Died',1:'Survived'})
df.head(50)

# Entendendo as variáveis
* Vamos verificar como as variáveis estão preenchidas a fim de corrigir possíveis problemas de preenchimento.

As funções a seguir nos ajudará com o Data Visualization, cruzando a variável-resposta 'Survived' com qualquer outra passada à função:

In [0]:
def Avalia_Taxa_Sobrevivencia(df, column):
    title_xt = pd.crosstab(df[column], df['Survived2'])
    print(pd.crosstab(df[column], df['Survived2'], margins=True))
    title_xt_pct = title_xt.div(title_xt.sum(1).astype(float), axis=0)
    
    title_xt_pct.plot(kind='bar', stacked=True, title='Taxa de Sobrevivência dos Passageiros', 
                      color= ['r', 'g'])
    plt.xlabel(column)
    plt.ylabel('Taxa de Sobrevivência')
    plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.05),shadow=True, ncol=2)
    plt.show()

def Catplot_Graph(x, y, hue= 'Survived2', col= None):
    plt.rcdefaults()
    g= sns.catplot(x= x, y= y, hue= hue, palette={'Died':'red','Survived':'blue'}, col= col, data=df, kind= 'bar', height=4, aspect=.7)
    plt.show()

## Variável 'Sex'

* Como está o preenchimento desta variável?

In [0]:
df_copia= df.copy()
df.Sex.value_counts()

Qual sua opinião sobre esse preenchimento?

* Algum problema?

Aqui temos vários problemas... Olhando para estes resultados, você concorda que 'male', 'm', 'MALE', M', 'mALE' e 'Men' se trata da mesma informação?

Da mesma forma, 'female', 'f', 'F', 'Female', 'fEMALE', 'Woman', 'w' e 'W' também se trata da mesma informação?

Então, vamos fazer o seguinte:

Toda vez que eu encontrar um desses valores: ['m', 'MALE', 'M', 'mALE', 'Men'], vou substituir por 'male';
Toda vez que eu encontrar um desses valores: ['f', 'F', 'Female', 'fEMALE', 'Woman', 'w', 'W'], vou substituit por 'female'.
O comando a seguir faz estas substituições:

Definindo o dicionário para fazermos as substituições dos valores inconsistentes:

In [0]:
dSex= {}
dSex.update(dict.fromkeys(['m', 'MALE', 'M', 'mALE', 'Men', 'male'], "male"))
dSex.update(dict.fromkeys(['f', 'F', 'Female', 'fEMALE', 'Woman', 'w', 'W', 'female'], 'female'))
dSex

In [0]:
df['Sex2']= df['Sex'].map(dSex)
df.Sex2.value_counts()

In [0]:
df.head()

**Atenção:** Os comandos abaixo são uma alternativa ao map() aplicado anteriormente para corrigir os atributos da variável 'Sex':

```
df['Sex2'] = df['Sex'].replace(['m', 'MALE', 'M', 'mALE', 'Men'], 'male')
df['Sex3'] = df['Sex2'].replace(['f', 'F', 'Female', 'fEMALE', 'Woman', 'w', 'W'], 'female') 
df.Sex3.value_counts()
```



Agora ok, temos algo que faz sentido para a variável 'Sex'.

Ok, de fato corrigimos os problemas de preenchimento da variável 'Sex'. então, vamos renomear nossa variável para o que tínhamos antes:

In [0]:
# Deleta as variáveis 'Sex':
df= df.drop(columns= ['Sex'], axis= 1)

# Renomea a variável auxiliar 'Sex2' para 'Sex':
df= df.rename(columns= {'Sex2': 'Sex'})

# Mostra os dados:
df.head()

In [0]:
sns.catplot(x="Sex", kind="count", data=df)

## Variável 'Cabin'

In [0]:
set(df['Cabin'])

Como podemos ver, trata-se de uma variável categórica com vários níveis. Portanto, vamos capturar somente a primeira letra da variável 'Cabin'. Para tal, vamos utilizar a função slice().

> slice() - Get substring from a given string using slice object;

A seguir, capturamos a primeira letra da variável 'Cabin':

In [0]:
# definindo a variável 'Deck' que representará a primeira letra da variável 'Cabin'
df["Deck"] = df["Cabin"].str.slice(0,1)
df.Deck.value_counts()

A seguir, vamos extrair a parte numérica da variável 'Cabin' usando Expressões Regulares:

In [0]:
# Import the library Regular Expressions
import re

In [0]:
# Primeiramente, usamos a função split() para separar o conteúdo da variável em colunas: 
new = df["Cabin"].str.split(" ", n = 3, expand = True) 
new.head()

Observe acima que o comando gera quantos splits da variável eu quiser. No entanto, por simplicidade, me interessa somente o primeiro split.

Agora, vou extrair o número do assento usando Expressões Regulares:

In [0]:
new2= new[0].str.extract('(\d+)')
new2.head()

Por fim, vou carregar esta informação ao dataframe df:

In [0]:
df["Seat"]= new2
df.head()

Por fim, excluir a variável 'Cabin':

In [0]:
df= df.drop(columns= ["Cabin"], axis=1, errors="ignore")

## Variável 'Embarked'

In [0]:
df.Deck.value_counts()

In [0]:
sns.catplot(x="Deck", kind="count", data=df)

Não vejo problemas com esta variável. Vamos em frente...

## Variável 'Pclass'

In [0]:
df.Pclass.value_counts()

Algum problema com esta variável?

In [0]:
sns.catplot(x="Pclass", kind="count", data=df)

## Variável 'Parch'

In [0]:
df.Parch.value_counts()

In [0]:
sns.catplot(x="Parch", kind="count", data=df)

## Variável 'SibSp'

In [0]:
df.SibSp.value_counts()

* Algum problema?

In [0]:
sns.catplot(x="SibSp", kind="count", data=df)

## Variável 'Fare'

Transformações: arredondar variável Fare.

In [0]:
df['Fare']= round(df['Fare'], 0)

In [0]:
df.plot.scatter('Age','Fare')

## Variável 'Age'

Transformações: arredondar variável Fare.

In [0]:
df['Age']= round(df['Age'], 0)

# Derivar atributos

## Variável 'MV_Age'

In [0]:
df.head()

Para construir a variável 'MV_Age', vamos utilizar a função pd.isna(). Por exemplo, o comando abaixo verifica se cada linha/observação da variável 'Age' é um NaN.

In [0]:
df['Age'].isna().sum()

A seguir, criamos uma variável auxiliar intitulada 'MV_AUX', que receberá 'True', caso 'Age' seja um valor válido e 'False' caso 'Age' seja NaN.

Veja abaixo:

In [0]:
df['MV_AUX']= df['Age'].isna()
df.head()

In [0]:
# Adiciona a nova coluna baseado no dicionario 
df['MV_Age'] = df['MV_AUX'].map({True: 1, False: 0})
df.head()

Deleta a variável auxiliar 'MV_AUX':

In [0]:
df= df.drop(columns= ['MV_AUX'], axis=1)

Nosso dataframe está assim:

In [0]:
df.head()

## Variável 'Title'

* Para fins de Data Manipulation, vamos capturar o tratamento dos passageiros contido na variável 'Name'. Ou seja, 'Mr.', 'Mrs.', 'Miss' e etc...

> Fonte: As funções get_title e title_map foram extraídas de https://www.kaggle.com/tjsauer/titanic-survival-python-solution

In [0]:
df.head()

In [0]:
def get_title(name):
    if '.' in name:
        return name.split(',')[1].split('.')[0].strip()
    else:
        return 'Unknown'

def title_map(title):
    if title in ['Mr', 'Ms']:
        return 1
    elif title in ['Master']:
        return 2
    elif title in ['Ms','Mlle','Miss']:
        return 3
    elif title in ["Mme", "Ms", "Mrs"]:
        return 4
    elif title in ["Jonkheer", "Don", "Sir", "the Countess", "Dona", "Lady"]:
        return 5
    elif title in ["Capt", "Col", "Major", "Dr", "Rev"]:
        return 6
    else:
        return 7

Captura o tratamento dos passageiros:

In [0]:
df['Title'] = df['Name'].apply(get_title).apply(title_map)  
set(df['Title']) # Esse comando mostra os NaN's da variável

Drop a coluna 'Name', pois não vamos mais precisar dela em nossas análises:

In [0]:
df= df.drop(columns= ["Name"], axis=1, errors="ignore")

Apresenta o conteúdo do dataframe:

In [0]:
df.head(3)

## Variável 'Family_Size'
* As variáveis SibSp e Parch estão relacionadas ao grupo familiar. Portanto, vamos criar a variável 'Family_Size', da seguinte forma:

In [0]:
df['Family_Size']= df['SibSp']+df['Parch']+1

A seguir, verificamos se esta operação produziu algum NaN.

In [0]:
df['Family_Size'].isnull().sum()

In [0]:
sns.catplot(x="Family_Size", kind="count", data=df)

In [0]:
set(df['Family_Size']) # Esse comando mostra os NaN's da variável

# Conclusão

In [0]:
df.head(50)

# Salvar cópia do dataframe tratado nesta fase

In [0]:
df.to_csv("df_3DP_FE1.csv", sep= ',', index = True, header=True)